In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from glob import glob

In [3]:
files = glob('gov.my/*.html')
len(files)

242

In [10]:
all_a = []
for f in tqdm(files):
    with open(f) as fopen:
        data = fopen.read()
    soup = BeautifulSoup(data, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href', '') for a_ in a if a_.get('href', '').endswith('.pdf')]
    all_a.extend(a)

100%|█████████████████████████████████████████| 242/242 [00:37<00:00,  6.42it/s]


In [12]:
all_a = list(set(all_a))
len(all_a)

39748

In [14]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [15]:
import mp

In [16]:
with open('all_a.json', 'w') as fopen:
    json.dump(all_a, fopen)

In [17]:
all_a_f = [(no, f) for no, f in enumerate(all_a)]

In [18]:
!mkdir pdf

In [21]:
def loop(files):
    files, _ = files
    for f in tqdm(files):
        i, f = f
        file = os.path.join('pdf', f'{i}.pdf')
        if os.path.exists(file):
            continue
        
        try:
            r = requests.get(f, timeout = 60)
            with open(file, 'wb') as fopen:
                fopen.write(r.content)
        except:
            pass

In [22]:
loop((all_a_f[:2], 0))

100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 34807.50it/s]


In [24]:
mp.multiprocessing(all_a_f, loop, cores = 10, returned = False)

100%|█████████████████████████████████████| 3974/3974 [1:46:01<00:00,  1.60s/it]
